# Choose single Z-slice


## preamble

1. imports
2. setup

> the contents of the preamble are reduncant with [00.1_pipeline_setup.ipynb](./00.1_pipeline_setup.ipynb)

In [9]:
# top level imports
from pathlib import Path
import os, sys
import json

import numpy as np

from typing import Union, List, Tuple, Any
# TODO:  prune the imports.. this is the big set for almost all organelles
# # function for core algorithm
from scipy import ndimage as ndi
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_slice_by_slice )

# # package for io 
from aicsimageio import AICSImage

import napari

from skimage.segmentation import clear_border
from skimage.measure import label

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                                                    list_image_files)
from infer_subc.core.img import *
from infer_subc.organelles import fixed_get_optimal_Z_image, get_optimal_Z_image

# NOTE:  these "constants" are only accurate for the testing MCZ dataset
from infer_subc.constants import (
    TEST_IMG_N,
    NUC_CH ,
    LYSO_CH ,
    MITO_CH ,
    GOLGI_CH ,
    PEROX_CH ,
    ER_CH ,
    LD_CH ,
    RESIDUAL_CH , 
    ALL_CHANNELS)

%load_ext autoreload
%autoreload 2

test_img_n = TEST_IMG_N

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# depricate this
# list_img_files = lambda img_folder,f_type: [os.path.join(img_folder,f_name) for f_name in os.listdir(img_folder) if f_name.endswith(f_type)]
img_file_list = list_image_files(data_path,im_type)

test_img_name = img_file_list[test_img_n]
test_img_name

'/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.czi'

In [5]:
img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


## CHOOZE Z-SLICE

We need to chooze which Z-slice to analyze for subsequent organelle segmentation.  _A priori_ we expect that the other organelles are ***NOT*** _inside_ of the nucleus, and we want to chooze a Z with minimal overlap between the nucleus and other organelles.  For now we will also assume that a majority of the signals are attributable to a single well marked cell.


In the future we might develop an optimization procedure to resample along an arbitrary plane such that the total florescence signal is maximized and the overlap on nuclei with other organelles.   We may also want to limit ourselves to a single neuron / cellmask.

Maria Clara's description of how she choose Z-slices:  "I do choose the slice which it contains all the organelle (Nuclei and Golgi are the most difficult one since show high polarity) and that it would contain the highest density of them (you can see trough the slices that some organelle show high density in specific planes)"


In [6]:
# get nuclei and normalize...
# median filter in 2D / convert to float 0-1.   get rid of the "residual"

nuclei = select_channel_from_raw(img_data, 0)

nuclei = min_max_intensity_normalization(nuclei)

med_filter_size = 4   
# structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
nuclei = median_filter_slice_by_slice(  nuclei,
                                                                size=med_filter_size  )

gaussian_smoothing_sigma = 1.34
nuclei = image_smoothing_gaussian_slice_by_slice(   nuclei, sigma=gaussian_smoothing_sigma )
threshold_value_log = threshold_li_log(nuclei)

thresh_factor = 0.9 #from cellProfiler
thresh_min = .1
thresh_max = 1.
threshold = min( max(threshold_value_log*thresh_factor, thresh_min), thresh_max)

struct_obj = nuclei > threshold



In [ ]:

###################
# mask everything and find most intense flourescence NOT in the nuclei
ch_to_agg = ( LYSO_CH ,
                        MITO_CH ,
                        GOLGI_CH ,
                        PEROX_CH ,
                        ER_CH ,
                        LD_CH )

total_florescence = img_data[ch_to_agg,].astype( np.double ).sum(axis=0)
print(total_florescence.shape)
total_ = total_florescence
total_[struct_obj] =0 
optimal_Z = total_.sum(axis=(1,2)).argmax()


Now make some functions to handle the procedure.  Note that we have "name mangled" the functions below by prepending the underscore '_' in order to test the local (mangled) version side-by-side with the library version which has been copied to the `infer_subc` library.

In [43]:

# this is redundant with  weighted_aggregate defined in cellmask.py / nbs
def _aggregate_signal_channels(img_in:np.ndarray, chs:Union[List, Tuple], ws:Union[List, Tuple, Any]= None) -> np.ndarray:
    """ 
    return a weighted sum of the image across channels

    img_in:  
        np.ndarray  [ch,z,x,y]
    chs: 
        list/tuple of channels to aggregate
    ws: 
        list/tuple/ of weights for aggregation
    """

    n_chan = len(chs)
    if n_chan <= 1:
        return img_in[chs]

    if ws is None:
        ws = n_chan*[1.]
    img_out = np.zeros_like(img_in[0]).astype(np.double)
    for w,ch in zip(ws,chs):
        img_out += w*img_in[ch]
    return img_out
    #return img_in[ch_to_agg,].astype( np.double ).sum(axis=0)


def _choose_agg_signal_zmax(img_in,chs,ws=None,mask=None):
    """ 
    return z the maximum signal for the aggregate signal

    img_in:  np.ndarray  [ch,z,x,y]
    chs: list of channels to aggregate
    ws: list of weights for aggregation
    mask: mask for img_in

    returns z with maximum signal
    """
    total_florescence_ = aggregate_signal_channels(img_data, chs)
    if mask is not None:
        total_florescence_[mask] = 0.
    return total_florescence_.sum(axis=(1,2)).argmax()


optimal_Z = _choose_agg_signal_zmax(img_data,ch_to_agg)



In [44]:


def _select_channel_from_raw(img_in:np.ndarray, ch:Union[int, Tuple[int]]) -> np.ndarray:
    """" 
    Parameters
    ------------
    img_in:

    ch :
        channel to extract.

    Returns
    -------------
        ndarray
    """
    return img_in[ch]


def _select_z_from_raw(img_in: np.ndarray, z_slice: Union[int, Tuple[int]]) -> np.ndarray:
    """
    Procedure to infer _best_ Zslice from linearly unmixed input with fixed parameters
    """
    if isinstance(z_slice, int):
        z_slice = [z_slice]
    else:
        z_slice = list(z_slice)

    return img_in[:, z_slice, :, :]

In [45]:
    
def _apply_log_li_threshold(img_in, thresh_factor=1.0, thresh_min=None, thresh_max=None):
    """return a binary mask after applying a log_li threshold
    
    Parameters
    ------------
    img_in: 
        input image
    thresh_factor:  
        scaling value for threshold
    thresh_min:   
        None (default) or min
    thresh_max:  
        None (default) or max

    Returns
    -------------
        img_in > threshold boolean ndarray
    """
    #struct_obj = struct_img > filters.threshold_li(struct_img)
    threshold_value_log = threshold_li_log(img_in)
    threshold = threshold_value_log*thresh_factor

    if thresh_min is not None:
        threshold = max(threshold, thresh_min)
    if thresh_max is not None:
        threshold = min(threshold, thresh_max)
    return img_in > threshold



In [46]:

def _find_optimal_Z(raw_img:np.ndarray, nuc_ch:int, ch_to_agg:Tuple[int]) -> int:
    """
    Procedure to infer _best_ Zslice  from linearly unmixed input.

    Parameters
    ------------
    raw_img: np.ndarray
        a ch,z,x,y - image containing florescent signal

    nuclei_ch: int
        holds the needed parameters

    nuclei_ch: int
        holds the needed parameters

    Returns
    -------------
    opt_z:
        the "0ptimal" z-slice which has the most signal intensity for downstream 2D segmentation    
    """

    # median filter in 2D / convert to float 0-1.   get rid of the "residual"
    struct_img = _select_channel_from_raw(raw_img, nuc_ch)

    struct_img = min_max_intensity_normalization(struct_img)
    med_filter_size = 4   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    nuclei = median_filter_slice_by_slice( struct_img,
                                                                    size=med_filter_size  )

    gaussian_smoothing_sigma = 1.34
    nuclei = image_smoothing_gaussian_slice_by_slice(  nuclei,
                                                                                                sigma=gaussian_smoothing_sigma
                                                                                                )
    thresh_factor = 0.9 #from cellProfiler
    thresh_min = .1
    thresh_max = 1.
    struct_obj = _apply_log_li_threshold(nuclei, thresh_factor=thresh_factor, thresh_min=thresh_min, thresh_max=thresh_max)
    optimal_Z = _choose_agg_signal_zmax(raw_img,ch_to_agg, mask=struct_obj)

    return optimal_Z


In [47]:
def _fixed_find_optimal_Z(img_data):
    """
    Procedure to infer _best_ Zslice  from linearly unmixed input with fixed parameters
    """
    nuc_ch = NUC_CH
    ch_to_agg = ( LYSO_CH ,
                            MITO_CH ,
                            GOLGI_CH ,
                            PEROX_CH ,
                            ER_CH ,
                            LD_CH )
    return _find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

In [48]:

def _get_optimal_Z_image(img_data: np.ndarray, nuc_ch: int, ch_to_agg: Tuple[int]) -> np.ndarray:
    """
    Procedure to infer _best_ Zslice from linearly unmixed input

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels

    nuc_ch: int
        channel with nuclei signal

    ch_to_agg: Tuple[int]
        tuple of channels to aggregate for selecting Z

    Returns
    -------------
    np.ndarray
        image array with single selected Z-slice   (Channels, 1, X, Y)

    """
    optimal_Z = _find_optimal_Z(img_data, nuc_ch, ch_to_agg)
    return _select_z_from_raw(img_data, optimal_Z)


def _fixed_get_optimal_Z_image(img_data: np.ndarray) -> np.ndarray:
    """
    Procedure to infer _best_ Zslice from linearly unmixed input with fixed parameters
    """
    optimal_Z = _fixed_find_optimal_Z(img_data)
    return _select_z_from_raw(img_data, optimal_Z)

In [49]:
optimal_Z = _fixed_find_optimal_Z(img_data) 


In [50]:
# test the module functions
from infer_subc.organelles import fixed_find_optimal_Z, fixed_get_optimal_Z_image

optimal_Z = fixed_find_optimal_Z(img_data) 
single_Z_img = fixed_get_optimal_Z_image(img_data) 


choosing _optimal_ z-slice::: 8
choosing _optimal_ z-slice::: 8


## helper function to add function entries
### interface to `organelle-segmenter-plugin` repo 
These helpers encode the workflow particulars for creating widgets and defining/executing workflows.    This function will be added to `infer_subc.organelles_config.helper.py` 

The gist of things is we need to add all functions to "all_functions.json" so workflows can access them.  i.e. via the `organelle-segmenter-plugin`

In [51]:
from infer_subc.utils.directories import Directories

def _add_function_spec_to_widget_json( function_name, function_dict, json_file_name = "all_functions.json", overwrite=False):
    """ helper function to compose / update list of functions for Workflows
    """
    # read all_functions.json into dict
    path = Directories.get_structure_config_dir() / json_file_name
    try:
        with open(path) as file:
            obj = json.load(file)
    except: #Exception as ex:
        print(f"file {path} not found")
        return 
        
    # add function entry
    if function_name in obj.keys():
        print(f"function {function_name} is already in {json_file_name}")
        if not overwrite:
            return(0)
    
    obj[function_name] = function_dict    # write updated all_functions.json
        
    # re-write file
    with open(path, "w") as file:
        json.dump(obj, file, indent=4, sort_keys=False)
        
    return(1)


In [52]:
fixed_find_optimal_Z_dict =  {
        "name": "find the optimal Z slice (fixed parameters)",
        "python::module": "infer_subc.organelles",
        "python::function": "fixed_find_optimal_Z",
        "parameters": None
        }

fixed_find_optimal_Z_dict

{'name': 'find the optimal Z slice (fixed parameters)',
 'python::module': 'infer_subc.organelles',
 'python::function': 'fixed_find_optimal_Z',
 'parameters': None}

In [53]:
fixed_get_optimal_Z_img_dict =  {
        "name": "extract optimal Z slice (fixed parameters)",
        "python::module": "infer_subc.organelles",
        "python::function": "fixed_get_optimal_Z_image",
        "parameters": None
        }

fixed_get_optimal_Z_img_dict

{'name': 'extract optimal Z slice (fixed parameters)',
 'python::module': 'infer_subc.organelles',
 'python::function': 'fixed_get_optimal_Z_image',
 'parameters': None}

In [54]:
from infer_subc.organelles_config.helper import add_function_spec_to_widget_json

add_function_spec_to_widget_json("fixed_find_optimal_Z",fixed_find_optimal_Z_dict, overwrite=True)
add_function_spec_to_widget_json("fixed_get_optimal_Z_img",fixed_get_optimal_Z_img_dict, overwrite=True)

function fixed_find_optimal_Z is already in all_functions.json
overwriting  fixed_find_optimal_Z
function fixed_get_optimal_Z_img is already in all_functions.json
overwriting  fixed_get_optimal_Z_img


1

In [55]:
from infer_subc.constants import ALL_CHANNELS
_select_channel_from_raw_dict =  {
        "name": "select a channel ",
        "python::module": "infer_subc.core.img",
        "python::function": "select_channel_from_raw",
        "parameters": {
            "chan": {
                "data_type": "int",
                "options": ALL_CHANNELS,
                "widget_type": "drop-down"
            }
        }
}

add_function_spec_to_widget_json("select_channel_from_raw",_select_channel_from_raw_dict )

function select_channel_from_raw is already in all_functions.json


0

In [56]:
ch_to_agg = (1,2,3,4,5,6)
nuc_ch = 0

In [57]:
optimal_Z = _find_optimal_Z(img_data, nuc_ch, ch_to_agg) 


-------------
## SUMMARY

The above demonstrates choosing an "optimal" Z-slice from our 3D multichannel input. 

### NEXT: INFER SOMA

proceed to [01_infer_cellmask_fromaggr.ipynb](./01_infer_cellmask_fromaggr.ipynb)
